<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [2]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [3]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

In [4]:
# Examining the data
kobe.shape

(1558, 645)

In [13]:
# Seeing the broad datastructure
kobe.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [23]:
kobe.isnull().sum()

shots_made                                 0
away_game                                  0
season_opponent:atl:1996-97                0
season_opponent:atl:1997-98                0
season_opponent:atl:1999-00                0
                                          ..
action_type:turnaround_finger_roll_shot    0
action_type:turnaround_hook_shot           0
action_type:turnaround_jump_shot           0
season_game_number                         0
career_game_number                         0
Length: 645, dtype: int64

**Note**: Each column pertains to a statistic from a particular game (row)

In [20]:
# Change column names to lower-case
kobe.columns = kobe.columns.str.lower()

In [26]:
list(kobe.columns)

['shots_made',
 'away_game',
 'season_opponent:atl:1996-97',
 'season_opponent:atl:1997-98',
 'season_opponent:atl:1999-00',
 'season_opponent:atl:2000-01',
 'season_opponent:atl:2001-02',
 'season_opponent:atl:2002-03',
 'season_opponent:atl:2003-04',
 'season_opponent:atl:2004-05',
 'season_opponent:atl:2005-06',
 'season_opponent:atl:2006-07',
 'season_opponent:atl:2007-08',
 'season_opponent:atl:2008-09',
 'season_opponent:atl:2009-10',
 'season_opponent:atl:2010-11',
 'season_opponent:atl:2011-12',
 'season_opponent:atl:2012-13',
 'season_opponent:atl:2013-14',
 'season_opponent:atl:2014-15',
 'season_opponent:atl:2015-16',
 'season_opponent:bkn:2012-13',
 'season_opponent:bkn:2015-16',
 'season_opponent:bos:1996-97',
 'season_opponent:bos:1997-98',
 'season_opponent:bos:1999-00',
 'season_opponent:bos:2001-02',
 'season_opponent:bos:2002-03',
 'season_opponent:bos:2003-04',
 'season_opponent:bos:2004-05',
 'season_opponent:bos:2005-06',
 'season_opponent:bos:2006-07',
 'season_op

In [28]:
opponents.head()

,season_opponent:atl:1996-97,season_opponent:atl:1997-98,season_opponent:atl:1999-00,season_opponent:atl:2000-01,season_opponent:atl:2001-02,season_opponent:atl:2002-03,season_opponent:atl:2003-04,season_opponent:atl:2004-05,season_opponent:atl:2005-06,season_opponent:atl:2006-07,...,season_opponent:was:2005-06,season_opponent:was:2006-07,season_opponent:was:2007-08,season_opponent:was:2008-09,season_opponent:was:2009-10,season_opponent:was:2010-11,season_opponent:was:2011-12,season_opponent:was:2012-13,season_opponent:was:2014-15,season_opponent:was:2015-16
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [43]:
# Defining X
X = kobe.drop(columns=['shots_made'])
print(type(X))
X.head()

<class 'pandas.core.frame.DataFrame'>


,away_game,season_opponent:atl:1996-97,season_opponent:atl:1997-98,season_opponent:atl:1999-00,season_opponent:atl:2000-01,season_opponent:atl:2001-02,season_opponent:atl:2002-03,season_opponent:atl:2003-04,season_opponent:atl:2004-05,season_opponent:atl:2005-06,...,action_type:tip_layup_shot,action_type:tip_shot,action_type:turnaround_bank_shot,action_type:turnaround_fadeaway_bank_jump_shot,action_type:turnaround_fadeaway_shot,action_type:turnaround_finger_roll_shot,action_type:turnaround_hook_shot,action_type:turnaround_jump_shot,season_game_number,career_game_number
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [45]:
# Defining y
y = kobe['shots_made']
print(type(y))
y.head()

<class 'pandas.core.series.Series'>


0    0.0
1    0.0
2    2.0
3    2.0
4    0.0
Name: shots_made, dtype: float64

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
# Split data into training set and a holdout set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)
print(f'Train size: {len(X_train)}, Test size: {len(X_test)}')

Train size: 1168, Test size: 390


In [11]:
from sklearn.preprocessing import StandardScaler

In [84]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [85]:
lr = LinearRegression()

In [86]:
lr_scores = cross_val_score(lr, Z_train, y_train, cv=10)
print(lr_scores)
print(lr_scores.mean())

[-1.89153068e+28 -3.46262484e+28 -7.26682307e+27 -5.10141902e+27
 -1.91623310e+28 -2.06638130e+28 -1.58596737e+28 -8.94217394e+27
 -2.91047751e+28 -2.02620062e+28]
-1.7990457014567751e+28


---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


In [87]:
from sklearn.linear_model import RidgeCV

In [88]:
r_alphas = np.logspace(0, 5, 100)

In [89]:
ridge_cv = RidgeCV(alphas=r_alphas, scoring='r2',
                  cv=10).fit(Z_train, y_train)

In [96]:
opt_alpha = ridge_cv.alpha_
opt_alpha

1204.5035402587823

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [90]:
print(ridge_cv.score(Z_train, y_train))
print(ridge_cv.score(Z_test, y_test))

0.7781911372916419
0.5937116596353558


---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [ ]:
from sklearn.linear_model import LassoCV

In [108]:
l_alphas = np.logspace(-3, 5, 100)
lasso_cv = LassoCV(alphas=l_alphas, cv=10,
                  max_iter=50000).fit(Z_train, y_train)

In [111]:
opt_alpha2 = lasso_cv.alpha_
opt_alpha2

0.08697490026177834

---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [109]:
print(lasso_cv.score(Z_train, y_train))
print(lasso_cv.score(Z_test, y_test))

0.7087948768391135
0.6466825189527826


---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [117]:
features = X.columns

In [124]:
lasso_coefs = pd.Series(lasso_cv.coef_, index=features).sort_values(ascending=False)

In [129]:
len(lasso_coefs[lasso_coefs==0])/len(lasso_coefs)

0.8369565217391305

In [132]:
lasso_coefs[lasso_coefs!=0]

combined_shot_type:jump_shot              1.194189
shot_type:2pt_field_goal                  0.991120
shot_zone_basic:restricted_area           0.476903
shot_zone_basic:above_the_break_3         0.223081
combined_shot_type:dunk                   0.219362
                                            ...   
action_type:driving_floating_jump_shot   -0.077839
season_opponent:bos:2015-16              -0.088432
season:2015-16                           -0.091869
action_type:layup_shot                   -0.116176
action_type:jump_shot                    -0.210480
Length: 105, dtype: float64

**Note**: (1) Number of jump shots, (2) number of 2-point field goals, (3) shots taken in the restricted area and (4) above the break (3), (5) Number of dunks and (5) % of shots comprising of jump shots

---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [134]:
from sklearn.linear_model import ElasticNetCV

In [143]:
# Set up list of alphas to check
enet_alphas = np.linspace(0.1, 1.0, 100) # Return evenly-spaced numbers over a specified interval
enet_l1_ratios = np.linspace(0.5, 1.0, 100) # Return evenly-spaced numbers over a specified interval

# Fit model using optimal alpha
enet_cv = ElasticNetCV(alphas=enet_alphas, l1_ratio=enet_l1_ratios, cv=10).fit(Z_train, y_train)

In [144]:
# Find optimal alpha
enet_cv.alpha_

0.1

In [145]:
# Find optimal l1 ratio
enet_cv.l1_ratio_

0.7828282828282829

---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

In [146]:
# Fit model using optimal alpha
print(enet_cv.score(Z_train, y_train))
print(enet_cv.score(Z_test, y_test))

0.7201541170339383
0.641116996735224


---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [ ]:
# A: Maybe a jointplot?